In [3]:
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
import requests

0. TOOL CREATION

In [12]:
@tool
def multiply(a:int, b: int)->int:
    """Given two numbers a and b, this tool return product of a and b """
    return a*b

print(multiply.invoke({'a':4, 'b':7}))

28


In [25]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Given two numbers a and b, this tool return product of a and b
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


1. TOOL BINDING

In [7]:
llm=ChatOpenAI()

llm_with_tools=llm.bind_tools([multiply]) #this is tool binding. NOT all LLMs have tool binding capability

2. TOOL CALLING

In [8]:
llm_with_tools.invoke("How are you?")

AIMessage(content="I'm here and ready to help you! What can I assist you with today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 61, 'total_tokens': 79, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BhQomWNjiy2i1eDuzz8wELYfQxhaJ', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--1c1ce311-1247-46d7-ba15-dcf10198227c-0', usage_metadata={'input_tokens': 61, 'output_tokens': 18, 'total_tokens': 79, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [26]:
llm_with_tools.invoke("what is 8 times 10")
# there is no content but a tool call 

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ZbP5EIEWhs8ti7Z6wcP0ti1I', 'function': {'arguments': '{"a":8,"b":10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 64, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BhRDnFhaC2PoxLRBRrr4nqJGKAcLX', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--1bd511ed-522b-4e52-a4c1-7ca5b21ee320-0', tool_calls=[{'name': 'multiply', 'args': {'a': 8, 'b': 10}, 'id': 'call_ZbP5EIEWhs8ti7Z6wcP0ti1I', 'type': 'tool_call'}], usage_metadata={'input_tokens': 64, 'output_tokens': 17, 'total_tokens': 81, 'input_token_details': {'audio': 0, 'cache_read': 0}, 

In [27]:
llm_with_tools.invoke("what is 8 times 10").tool_calls

[{'name': 'multiply',
  'args': {'a': 8, 'b': 10},
  'id': 'call_PMcB0oCs7gFe8kqSDTdN4TYr',
  'type': 'tool_call'}]

3. TOOL EXECUTION

In [29]:
result=llm_with_tools.invoke("what is 8 times 10").tool_calls[0]
args=result['args']
print(args)
multiply.invoke(args)

{'a': 8, 'b': 10}


80

In [30]:
# we can send the entire tool call to the tool
result=llm_with_tools.invoke("what is 8 times 10").tool_calls[0]
multiply.invoke(result)

ToolMessage(content='80', name='multiply', tool_call_id='call_N80M9o18jMBFPCIYdupCd0Vf')

4. FEDDING IT TO LLM

        to give this output of the tool to the LLM to give a response, we need to oraganize things little bit

In [31]:
messages=[]
query=HumanMessage("what is 8 times 10")
messages.append(query)

In [34]:
result=llm_with_tools.invoke(query.content)
messages.append(result)

In [39]:
messages

[HumanMessage(content='what is 8 times 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_lI0zzfA2mWhd0kefaVEuV7zB', 'function': {'arguments': '{"a":8,"b":10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 64, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BhRK17nmtpnfAARhD6NE3fW87IYOv', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--acbec826-7c02-42bd-a78e-62ad211b7fb9-0', tool_calls=[{'name': 'multiply', 'args': {'a': 8, 'b': 10}, 'id': 'call_lI0zzfA2mWhd0kefaVEuV7zB', 'type': 'tool_call'}], usage_metadata={'input_tokens': 64, 'outp

In [41]:
tool_result=multiply.invoke(result.tool_calls[0])

In [42]:
tool_result

ToolMessage(content='80', name='multiply', tool_call_id='call_lI0zzfA2mWhd0kefaVEuV7zB')

In [43]:
messages.append(tool_result)
messages

[HumanMessage(content='what is 8 times 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_lI0zzfA2mWhd0kefaVEuV7zB', 'function': {'arguments': '{"a":8,"b":10}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 64, 'total_tokens': 81, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BhRK17nmtpnfAARhD6NE3fW87IYOv', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--acbec826-7c02-42bd-a78e-62ad211b7fb9-0', tool_calls=[{'name': 'multiply', 'args': {'a': 8, 'b': 10}, 'id': 'call_lI0zzfA2mWhd0kefaVEuV7zB', 'type': 'tool_call'}], usage_metadata={'input_tokens': 64, 'outp

In [45]:
llm_with_tools.invoke(messages)
# sending all the conversation history to the LLM

AIMessage(content='The product of 8 and 10 is 80.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 89, 'total_tokens': 102, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BhRMrUx4Jb3XD0ml2IKdkT9OgQCYl', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--f7439763-a354-4016-a0bd-176854e0f703-0', usage_metadata={'input_tokens': 89, 'output_tokens': 13, 'total_tokens': 102, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})